<a href="https://colab.research.google.com/github/amasnaoui/Classification_damaged_not_damaged_car/blob/main/DenseNet121_classification_acc_89_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import necessary librabry
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [ ]:
#defining the base, train and validation directory path
base_dir = '/content/drive/MyDrive/new_cars_dataset/'
train_dir = os.path.join(base_dir, 'training')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
#defining the damage and whole , train, validation and test directory
train_damage_dir = os.path.join(train_dir, '00-damage')
train_whole_dir = os.path.join(train_dir, '01-whole')
validation_damage_dir = os.path.join(validation_dir, '00-damage')
validation_whole_dir = os.path.join(validation_dir, '01-whole')
test_damage_dir = os.path.join(test_dir, '00-damage')
test_whole_dir = os.path.join(test_dir, '01-whole')

In [ ]:
#data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range = 40,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)
val_datagen = ImageDataGenerator(rescale=1./255)
testt_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(150, 150), 
        batch_size=20,
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')
testt_generator = testt_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

In [ ]:
#defining model
from tensorflow.keras import Model 
from tensorflow.keras.applications import DenseNet121   

base_model = DenseNet121(input_shape = (150, 150, 3),  include_top = False, weights = 'imagenet') 

In [ ]:
x=   tf.keras.layers.Flatten()(base_model.output)
x=   tf.keras.layers.Dense(512, activation='relu')(x)
x=   tf.keras.layers.Dense(1, activation='sigmoid')(x) 

model= Model(base_model.input, x)

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['Accuracy','Precision','Recall'])

In [ ]:
#training the model
history = model.fit(
      train_generator,
      epochs=20,
      validation_data=validation_generator,
      verbose=1)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['Accuracy']
val_acc = history.history['val_Accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['precision']
val_acc = history.history['val_precision']
loss = history.history['recall']
val_loss = history.history['val_recall']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
scores = model.evaluate(testt_generator, verbose = 1)
print("Accuracy : %.2f%%" % (scores[1]*100))

In [ ]:
#weights saving
model.save("/content/drive/MyDrive/new_cars_dataset/models/89%_DenseNet121_classifier.h5")

In [ ]:
# from keras.models import load_model
# new_model = load_model("/content/drive/MyDrive/new_cars_dataset/models/DenseNet121_classifier_90%.h5")

In [ ]:
# dir_path = '/content/drive/MyDrive/clean data/scrap_didier.be/'

# j = 0
# for sub_dir in os.listdir(dir_path):
#   if j < 4:
#     for i in os.listdir(dir_path + sub_dir):
#       img = image.load_img(dir_path + sub_dir + '//' + i, target_size=(150,150))
#       plt.imshow(img)
#       plt.show()

#       X = image.img_to_array(img)
#       X = np.expand_dims(X,axis=0)
#       images = np.vstack([X])
#       val = model.predict(images)
#       if (val == 0).any() :
#         print("damaged car")
#       else :
#         print("Not damaged car")
#     j += 1

In [ ]:
dir_path = '/content/drive/MyDrive/test/'

for i in os.listdir(dir_path):
  img = image.load_img(dir_path + '//' + i, target_size=(150,150))
  plt.imshow(img)
  plt.show()

  X = image.img_to_array(img)
  X = np.expand_dims(X,axis=0)
  images = np.vstack([X])
  val = model.predict(images)
  if (val == 0).any() :
    print("damaged car")
  else :
    print("Not damaged car")